<a href="https://colab.research.google.com/github/aziesel/Coursera_Capstone/blob/master/ADSC_Capstone_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Battle of the Neighbourhoods, Week 1**

1) The question I seek to answer is: _where in Seattle might be best for a new coffee shop?_


Seattle is famous for its cafes and coffee culture. If an entrepreneur was looking to open a new coffee shop in a neighbourhood that is not already saturated, where might they look to open? In particular, this entrepreneur may wish to seek out neighbourhoods that have other attractions such as parks and shopping, where patrons may want to stop and enjoy a refreshing latte or americano before continuing on with their day. However, opening a café in a location with no or few other businesses is not desirable, as there will be less customer traffic in nthat area.

Seattle is located at 47.6°N, 122.3°W. The city is positioned between Puget Sound on its west and Lake Washington on its east, making for a geographically restricted city. According to [Wikipedia](https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Seattle), Seattle can be divided into a whopping 127 distinct neighbourhoods while the [city of Seattle](http://www.seattle.gov/neighborhoods/neighborhoods-and-districts) divides the city into seven municipal council districts. Our entrepreneur has done a little research and feels that districts 2, 3 and 7, which comprise one contiguous section of central Seattle uninterrupted by waterways, is the most fruitful ground for a new cafe. Summarizing, our territory to search now includes 51 neighbourhoods.

2) Now that we have restricted our search space to the neighbourhoods of central Seattle, we have a more manageable data set in which to search for our ideal cafe location. Our approach includes the following steps:



1.   Obtain geographic coordinates for each of 51 Seattle neighbourhoods
2.   Query FourSquare for venues in each neighbourhood
3.   Generate ranked venue lists of each neighbourhood
4.   Eliminate neighbourhoods where "coffee shop" or "café" is in the top 3 venues
5.   Of the remaining venues, rank them by their number of coffee shops and cafés
6.   Reduce our list to the 25% of neighbourhoods with the least number of coffee shops
7.   Rank those remaining neighbourhoods by total number of non-coffee shop venues
8.   Using FourSquare data, plot the locations of coffee shops in candidate neighbourhoods
9.   Use k-means clustering to find regions enriched for non-coffee shops



Following these steps and using the above described data, we should be able to find at least one location that might be appropriate for a new café in central Seattle.







# **Battle of the Neighbourhoods Week 2**

We will now implement the above outlined steps to find our ideal café locations. 

**1) Obtain geographic coordinates for each of 51 Seattle neighbourhoods**

Unfortunately the geographic centres of each neighbourhood is listed on a separate wikipedia page, so I've collected the data manually and stored it in a pandas dataframe. Of the original 51 neighbourhoods, 43 had coordinates available either in Wikipedia or Google; the remaining 8 neighbourhoods were either very small and/or encompassed by larger neighbourhoods (for example, North Beacon Hill is part of Beacon Hill).


In [ ]:
import pandas as pd

nb = pd. read_csv('seattle_nb.csv', header=None)
nb.columns = ["neighbourhood", "latitude", "longitude"]
nb


,neighbourhood,latitude,longitude
0,Lakewood/Seward Park,47.17000,-122.5200
1,Denny Blaine,47.37000,-122.1600
2,Rainier Beach,47.51000,-122.2600
3,New Holly,47.54000,-122.2900
4,Othello,47.54000,-122.2800
5,Brighton,47.55000,-122.2700
6,Rainier Valley,47.55000,-122.2900
7,Rainier Vista,47.56000,-122.2900
8,Beacon Hill,47.58000,-122.3150
9,Mount Baker,47.58000,-122.2900


**2) Query FourSquare for venues in each neighbourhood**

Now that we have the coordinates for neighbourhoods in our region of interest, we can collect information on existing venues in those neighbourhoods. We can use our FourSquare credentials that we set up earlier in this course to retrieve that info; as in previous assignments, we will look at 100 venues per neighbourhood in a 500 metre radius from the geographic centre of that neighbourhood.

In [ ]:
import requests
import json
from pandas.io.json import json_normalize

CLIENT_ID="KIEIJGM0YYHLGH3XXKDTDRNKB0YBRET5FQMKOKMWDA0XUJZS"
CLIENT_SECRET="XSC31TKQBQZCGA4HBAK0UBUE5BBMRDWHNHI50U1ATT0L4HZD"
VERSION='20180605'
LIMIT=100

def getNearbyVenues(names, lats, longs, radius=500):
  venues_list = []
  for name, lat, lng, in zip(names, lats, longs):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    lat,
    lng,
    radius,
    LIMIT)

    results = requests.get(url).json()["response"]['groups'][0]['items']
    venues_list.append([(
        name,
        lat,
        lng,
        v['venue']['name'],
        v['venue']['location']['lat'],
        v['venue']['location']['lng'],
        v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 'Latitude', 'Longitude', 'Venue', 'V latitude', 'V longitude', 'Category']

  return(nearby_venues)

area_venues = getNearbyVenues(names=nb['neighbourhood'], lats=nb['latitude'], longs=nb['longitude'])
area_venues.shape

(1469, 7)

**3) Generate ranked venue lists of each neighbourhood**
Now that we've collected up to 100 venues within 500 metres of the geographic centre of each of our neighbourhoods, lets tally and rank venues by their most frequently occurring venue type. We're interested especially in those neighbourhoods that don't have a lot of cafes and coffee shops.

In [ ]:
import numpy as np

nb_onehot = pd.get_dummies(area_venues[['Category']], prefix = "", prefix_sep = "")
nb_onehot['Neighbourhood'] = area_venues['Neighbourhood']
nb_fixed_cols = [nb_onehot.columns[-1]] + list(nb_onehot.columns[:-1])
nb_onehot = nb_onehot[nb_fixed_cols]

nb_grouped = nb_onehot.groupby('Neighbourhood').mean().reset_index()

num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
  try:
    columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
  except:
    columns.append('{}th Most Common Venue'.format(ind+1))

def return_most_common_venues(row, num_top_venues):
  row_categories = row.iloc[1:]
  row_categories_sorted = row_categories.sort_values(ascending=False)
  return row_categories_sorted.index.values[0:num_top_venues]

nb_venues_sorted = pd.DataFrame(columns=columns)
nb_venues_sorted['Neighbourhood'] = nb_grouped['Neighbourhood']
for ind in np.arange(nb_grouped.shape[0]):
  nb_venues_sorted.iloc[ind, 1:] = return_most_common_venues(nb_grouped.iloc[ind, :], num_top_venues)

nb_venues_sorted



,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Beacon Hill,Mexican Restaurant,Bakery,Gas Station,Filipino Restaurant,Bus Station,Scenic Lookout,Light Rail Station,Taco Place,Fried Chicken Joint,Café
1,Belltown,Breakfast Spot,Coffee Shop,Bar,Gym,Hotel,Bakery,Cruise Ship,Italian Restaurant,Seafood Restaurant,Sushi Restaurant
2,Briarcliff,Athletics & Sports,Trail,Park,Yoga Studio,Electronics Store,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant
3,Brighton,Plaza,Farmers Market,Art Gallery,Scenic Lookout,Yoga Studio,Dive Bar,Fast Food Restaurant,Falafel Restaurant,Eye Doctor,Ethiopian Restaurant
4,Broadmoor,Botanical Garden,Tourist Information Center,Park,Golf Course,Trail,Eastern European Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store
5,Broadway,Italian Restaurant,Coffee Shop,Trail,Restaurant,Dog Run,Salad Place,Bus Stop,Bubble Tea Shop,Garden,Bakery
6,Capitol Hill,Coffee Shop,Cocktail Bar,American Restaurant,Italian Restaurant,Bar,Yoga Studio,Mexican Restaurant,Japanese Restaurant,Bakery,Thai Restaurant
7,Cascade,Coffee Shop,Bar,Café,Dessert Shop,Convenience Store,Sporting Goods Shop,Speakeasy,Breakfast Spot,Sandwich Place,Salon / Barbershop
8,Central Area,Vietnamese Restaurant,Coffee Shop,Pizza Place,Performing Arts Venue,Chinese Restaurant,Ethiopian Restaurant,Food Court,Bus Stop,Bakery,BBQ Joint
9,Colman,Park,Scenic Lookout,Trail,Italian Restaurant,French Restaurant,Garden,New American Restaurant,Coffee Shop,Gym,Eastern European Restaurant


**4) Eliminate neighbourhoods where "coffee shop" or "café" is in the top 3 venues**
Just by looking at our ranked venues table, we can see a lot of these neighbourhoods have a lot of coffee shops. We will now remove any neighbourhood that has "Coffee Shop" or "Café" as one of its top three venues. We'll also make a list of coffee saturated neighbourhoods for later.

In [ ]:
nb_no_coffee = nb_venues_sorted[nb_venues_sorted['1st Most Common Venue'] != 'Coffee Shop']
nb_no_coffee = nb_no_coffee[nb_no_coffee['1st Most Common Venue'] != 'Café']

nb_no_coffee = nb_no_coffee[nb_no_coffee['2nd Most Common Venue'] != 'Coffee Shop']
nb_no_coffee = nb_no_coffee[nb_no_coffee['2nd Most Common Venue'] != 'Café']

nb_no_coffee = nb_no_coffee[nb_no_coffee['3rd Most Common Venue'] != 'Coffee Shop']
nb_no_coffee = nb_no_coffee[nb_no_coffee['3rd Most Common Venue'] != 'Café']

nb_coffee = nb_venues_sorted[~nb_venues_sorted.index.isin(nb_no_coffee.index)]

nb_no_coffee

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Beacon Hill,Mexican Restaurant,Bakery,Gas Station,Filipino Restaurant,Bus Station,Scenic Lookout,Light Rail Station,Taco Place,Fried Chicken Joint,Café
2,Briarcliff,Athletics & Sports,Trail,Park,Yoga Studio,Electronics Store,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant
3,Brighton,Plaza,Farmers Market,Art Gallery,Scenic Lookout,Yoga Studio,Dive Bar,Fast Food Restaurant,Falafel Restaurant,Eye Doctor,Ethiopian Restaurant
4,Broadmoor,Botanical Garden,Tourist Information Center,Park,Golf Course,Trail,Eastern European Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store
9,Colman,Park,Scenic Lookout,Trail,Italian Restaurant,French Restaurant,Garden,New American Restaurant,Coffee Shop,Gym,Eastern European Restaurant
10,Denny Blaine,Construction & Landscaping,Intersection,Yoga Studio,Dive Bar,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Eye Doctor,Ethiopian Restaurant
15,Harborview,Chinese Restaurant,Vietnamese Restaurant,Tea Room,Dance Studio,Sushi Restaurant,Museum,Garden,Poke Place,Thai Restaurant,Hotpot Restaurant
17,International District,Chinese Restaurant,Vietnamese Restaurant,Tea Room,Dance Studio,Sushi Restaurant,Museum,Garden,Poke Place,Thai Restaurant,Hotpot Restaurant
22,Madison Park,Bank,Korean Restaurant,Dry Cleaner,Coffee Shop,Ski Area,Ice Cream Shop,Track,Eastern European Restaurant,Donut Shop,Dumpling Restaurant
24,Magnolia,Breakfast Spot,Athletics & Sports,Restaurant,Trail,Yoga Studio,Eastern European Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store


**5) Of the remaining venues, rank them by their number of coffee shops and cafés.**

In [ ]:
#check each row for coffee shop/cafe in 4th, append to new dataframe, repeat for each column. any remaining row that hasn't been appended gets stuck to the end of the new dataframe.
nb_no_coffee_ordered = pd.DataFrame(columns=columns)
for i, row in nb_no_coffee.iterrows():
  if nb_no_coffee.loc[i,'4th Most Common Venue'] == 'Coffee Shop':
    nb_no_coffee_ordered = nb_no_coffee_ordered.append(nb_no_coffee.loc[i])
  if nb_no_coffee.loc[i,'4th Most Common Venue'] == 'Café':
    nb_no_coffee_ordered = nb_no_coffee_ordered.append(nb_no_coffee.loc[i])

for i, row in nb_no_coffee.iterrows():
  if nb_no_coffee.loc[i,'5th Most Common Venue'] == 'Coffee Shop':
    nb_no_coffee_ordered = nb_no_coffee_ordered.append(nb_no_coffee.loc[i])
  if nb_no_coffee.loc[i,'5th Most Common Venue'] == 'Café':
    nb_no_coffee_ordered = nb_no_coffee_ordered.append(nb_no_coffee.loc[i]) 

for i, row in nb_no_coffee.iterrows():
  if nb_no_coffee.loc[i,'6th Most Common Venue'] == 'Coffee Shop':
    nb_no_coffee_ordered = nb_no_coffee_ordered.append(nb_no_coffee.loc[i])
  if nb_no_coffee.loc[i,'6th Most Common Venue'] == 'Café':
    nb_no_coffee_ordered = nb_no_coffee_ordered.append(nb_no_coffee.loc[i]) 

for i, row in nb_no_coffee.iterrows():
  if nb_no_coffee.loc[i,'7th Most Common Venue'] == 'Coffee Shop':
    nb_no_coffee_ordered = nb_no_coffee_ordered.append(nb_no_coffee.loc[i])
  if nb_no_coffee.loc[i,'7th Most Common Venue'] == 'Café':
    nb_no_coffee_ordered = nb_no_coffee_ordered.append(nb_no_coffee.loc[i]) 

for i, row in nb_no_coffee.iterrows():
  if nb_no_coffee.loc[i,'8th Most Common Venue'] == 'Coffee Shop':
    nb_no_coffee_ordered = nb_no_coffee_ordered.append(nb_no_coffee.loc[i])
  if nb_no_coffee.loc[i,'8th Most Common Venue'] == 'Café':
    nb_no_coffee_ordered = nb_no_coffee_ordered.append(nb_no_coffee.loc[i]) 

for i, row in nb_no_coffee.iterrows():
  if nb_no_coffee.loc[i,'9th Most Common Venue'] == 'Coffee Shop':
    nb_no_coffee_ordered = nb_no_coffee_ordered.append(nb_no_coffee.loc[i])
  if nb_no_coffee.loc[i,'9th Most Common Venue'] == 'Café':
    nb_no_coffee_ordered = nb_no_coffee_ordered.append(nb_no_coffee.loc[i]) 

for i, row in nb_no_coffee.iterrows():
  if nb_no_coffee.loc[i,'10th Most Common Venue'] == 'Coffee Shop':
    nb_no_coffee_ordered = nb_no_coffee_ordered.append(nb_no_coffee.loc[i])
  if nb_no_coffee.loc[i,'10th Most Common Venue'] == 'Café':
    nb_no_coffee_ordered = nb_no_coffee_ordered.append(nb_no_coffee.loc[i]) 

leftover_check = nb_no_coffee.merge(nb_no_coffee_ordered, on='Neighbourhood')
leftovers = nb_no_coffee[(~nb_no_coffee.Neighbourhood.isin(leftover_check.Neighbourhood))]
nb_no_coffee_ordered = nb_no_coffee_ordered.append(leftovers)
nb_no_coffee_ordered

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Madison Park,Bank,Korean Restaurant,Dry Cleaner,Coffee Shop,Ski Area,Ice Cream Shop,Track,Eastern European Restaurant,Donut Shop,Dumpling Restaurant
31,Queen Anne,Pizza Place,Yoga Studio,Thai Restaurant,Mexican Restaurant,Cosmetics Shop,Coffee Shop,Playground,Restaurant,Bakery,Grocery Store
9,Colman,Park,Scenic Lookout,Trail,Italian Restaurant,French Restaurant,Garden,New American Restaurant,Coffee Shop,Gym,Eastern European Restaurant
28,Othello,Vietnamese Restaurant,Thai Restaurant,Asian Restaurant,Tex-Mex Restaurant,Cajun / Creole Restaurant,Bakery,Taco Place,Café,Mobile Phone Shop,Chinese Restaurant
0,Beacon Hill,Mexican Restaurant,Bakery,Gas Station,Filipino Restaurant,Bus Station,Scenic Lookout,Light Rail Station,Taco Place,Fried Chicken Joint,Café
2,Briarcliff,Athletics & Sports,Trail,Park,Yoga Studio,Electronics Store,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant
3,Brighton,Plaza,Farmers Market,Art Gallery,Scenic Lookout,Yoga Studio,Dive Bar,Fast Food Restaurant,Falafel Restaurant,Eye Doctor,Ethiopian Restaurant
4,Broadmoor,Botanical Garden,Tourist Information Center,Park,Golf Course,Trail,Eastern European Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store
10,Denny Blaine,Construction & Landscaping,Intersection,Yoga Studio,Dive Bar,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Eye Doctor,Ethiopian Restaurant
15,Harborview,Chinese Restaurant,Vietnamese Restaurant,Tea Room,Dance Studio,Sushi Restaurant,Museum,Garden,Poke Place,Thai Restaurant,Hotpot Restaurant


**6) Reduce our list to the 25% of neighbourhoods with the least number of coffee shops**

We can see that 10 of our neighbourhoods do not have coffee shops or cafés in their top ten attractions: Briarcliff, Brighton, Broadmoor, Denny Blaine, Montlake, Mount Baker, New Holly, Rainier Beach, Rainier Valley, and SoDo. Rather than take the bottom quartile of coffee-intensive neighbourhoods, we will proceed with these ten for our search.

**7) Rank those remaining neighbourhoods by total number of non-coffee shop venues**

For those ten neighbourhoods we will return to our FourSquare data. While they don't have coffee shops or cafés in their top ten attractions, they likely have *some* such shops. While we're at it, we'll collect the geographic coordinates of those shops.

In [ ]:
#for neighbourhoods in leftovers, get area_venue data, and tally incidence of coffee shop/cafe

candidate_nb = leftovers['Neighbourhood']

candidate_tally = []
coffee_shop_coords = []
venue_coords = []
venue_coords_only = []
for neighbourhood in candidate_nb:
  output = []
  tally = 0
  total_venues = 0
  for i, row in area_venues.iterrows():
    if area_venues.loc[i, 'Neighbourhood'] == neighbourhood:
      total_venues += 1
      if area_venues.loc[i, 'Category'] == 'Coffee Shop':
    #    print("found coffee shop in ", neighbourhood)
        tally += 1
        coords = [area_venues.loc[i, 'V latitude'], area_venues.loc[i, 'V longitude']]
        coffee_shop_coords.append(coords)
      if area_venues.loc[i, 'Category'] == 'Café':
     #   print("found cafe in ", neighbourhood)
        tally += 1
        coords = [area_venues.loc[i, 'V latitude'], area_venues.loc[i, 'V longitude']]
        coffee_shop_coords.append(coords)
      else:
     #   print("found other in ", neighbourhood)
        coords = [neighbourhood, area_venues.loc[i, 'V latitude'], area_venues.loc[i, 'V longitude']]
        venue_coords.append(coords)
        coords_only = [area_venues.loc[i, 'V latitude'], area_venues.loc[i, 'V longitude']]
        venue_coords_only.append(coords_only)
    output = [neighbourhood, tally, total_venues]
  candidate_tally.append(output)

sorted(candidate_tally, key = lambda x: x[1], reverse=True)



[['Harborview', 1, 35],
 ['International District', 1, 35],
 ['Rainier Valley', 1, 22],
 ['SoDo', 1, 44],
 ['Yesler Terrace', 1, 35],
 ['Briarcliff', 0, 3],
 ['Brighton', 0, 4],
 ['Broadmoor', 0, 5],
 ['Denny Blaine', 0, 2],
 ['Magnolia', 0, 4],
 ['Montlake', 0, 4],
 ['Mount Baker', 0, 6],
 ['New Holly', 0, 3],
 ['Rainier Beach', 0, 3]]

We can see from the bottom of our output list, we have several neighbourhoods with 0 coffee shops or cafés, but they also have few total venues. Since we want a bustling neighbourhood to open our business, we will favour neighbourhoods with many venues and few coffee options: Broadway, Harborview, International District, Rainier Valley, SoDo and Yesler Terrace.

**8) Using FourSquare data, plot the locations of competing coffee shops in candidate neighbourhoods**

We've got the coordinates of competing coffee shops now, as well as the coordinates for non-coffee shop venues in each candidate neighbourhood. We've also discovered, upon examining our data, that three of these neighbourhoods have the same 'centre', meaning that they each mapped the same coffee shop. Let's plot those competitors (in black) and venues (in blue):

In [ ]:
import folium

sea_map = folium.Map(location = [47.6, -122.3], zoom_start=12)

for nb, lat, long in venue_coords:
    if nb in ('Harborview', 'International District', 'SoDo', 'Yesler Terrace', 'Rainier Valley'):
      label = nb
      folium.CircleMarker(
        [lat, long],
        radius=5,
        color='Blue',
        fill=True,
        fill_color='#3186cc',
        parse_html=False).add_to(sea_map)

for lat, long in coffee_shop_coords:
  label = "Competition"
  folium.CircleMarker(
    [lat, long],
    radius=5,
    color='black',
    fill=True,
    fill_color='black').add_to(sea_map)


sea_map

**9) Use k-means clustering to find regions enriched for non-coffee shops**
Now that we have our coffee shops/cafés and non-coffee venues mapped, let's do k-means clustering to find an area in these candidate neighbourhoods that has a lot of other attractive venues. We'll use a k = 15; just because we have three neighbourhoods to consider now doesn't mean each neighbourhood will be one cluster of venues. We want to find the centroid of each of these, and one k per region should find the centroid. Venues are in blue, competing coffee shops are in black, and candidate locations for our new coffee shop are indicated in red.


In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

#only use venues from the neighbourhoods with lots of venues
final_nb_venues = []
for row in venue_coords_only:
  if nb in ('Harborview', 'International District', 'SoDo', 'Yesler Terrace', 'Rainier Valley'):
    final_nb_venues.append(row)

kmeans = KMeans(n_clusters = 20, random_state = 0).fit(final_nb_venues)
labels = kmeans.predict(final_nb_venues)
centroids = kmeans.cluster_centers_
shop_locations = [centroids[i] for i in labels]

#shop_locations = pd.DataFrame(kmeans.cluster_centers_[find_centroids['cluster ID']], columns=['latitude', 'longitude'])

sea_map = folium.Map(location = [47.6, -122.3], zoom_start=12)

for nb, lat, long in venue_coords:
    if nb in ('Harborview', 'International District', 'SoDo', 'Yesler Terrace', 'Rainier Valley'):
      label = nb
      folium.CircleMarker(
        [lat, long],
        radius=5,
        color='Blue',
        fill=True,
        fill_color='#3186cc',
        parse_html=False).add_to(sea_map)

for lat, long in coffee_shop_coords:
  label = "Competition"
  folium.CircleMarker(
    [lat, long],
    radius=5,
    color='black',
    fill=True,
    fill_color='black').add_to(sea_map)

for lat, lng in shop_locations:
  label = "Candidate location"
  folium.CircleMarker(
      [lat, lng],
      radius=5,
      color='Red',
      fill=True,
      fill_color='red').add_to(sea_map)
sea_map



In [ ]:
labels

array([16, 16, 16,  9,  9,  9,  9, 10, 10, 10, 10, 10,  3,  3, 18, 18, 18,
       18, 18,  1,  8,  1,  8,  8,  8,  1,  1,  1,  1, 15,  8, 15, 15,  1,
        1, 15, 15, 15,  8,  1,  8, 15, 15, 15,  1, 15, 15,  8, 18, 18, 18,
       18, 18,  1,  8,  1,  8,  8,  8,  1,  1,  1,  1, 15,  8, 15, 15,  1,
        1, 15, 15, 15,  8,  1,  8, 15, 15, 15,  1, 15, 15,  8, 16,  2, 16,
        2,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6, 14, 14, 14,  0,  0,  0,
       17, 11,  4,  4, 11, 11, 11,  4,  4,  4,  4,  4,  4, 17, 17,  4, 11,
       11, 11, 11, 11,  7,  7,  7, 12, 12,  7, 12,  7, 13, 13,  7, 13,  7,
       13, 12,  7,  7, 13, 12, 13,  7, 19, 19, 19,  7,  7,  7,  7,  7,  7,
        7, 12,  7, 12,  7, 13,  7,  7,  7,  7,  7,  7,  7,  7, 18, 18, 18,
       18, 18,  1,  8,  1,  8,  8,  8,  1,  1,  1,  1, 15,  8, 15, 15,  1,
        1, 15, 15, 15,  8,  1,  8, 15, 15, 15,  1, 15, 15,  8],
      dtype=int32)

In [ ]:
print(kmeans)

NameError: ignored